In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import boxball_loader as bbl
import baseball_stats_utils as bsu

In [2]:
bat = bbl.load_batting(coalesce_type=bbl.CoalesceMode.PLAYER_SEASON_LEAGUE)
bat

ab   r    h  _2b  _3b  hr   rbi    sb   cs  bb    so  \
player_id yr   lg_id                                                          
aardsda01 2004 NL       0   0    0    0    0   0   0.0   0.0  0.0   0   0.0   
          2006 NL       2   0    0    0    0   0   0.0   0.0  0.0   0   0.0   
          2007 AL       0   0    0    0    0   0   0.0   0.0  0.0   0   0.0   
          2008 AL       1   0    0    0    0   0   0.0   0.0  0.0   0   1.0   
          2009 AL       0   0    0    0    0   0   0.0   0.0  0.0   0   0.0   
...                   ...  ..  ...  ...  ...  ..   ...   ...  ...  ..   ...   
zwilldu01 1915 FL     548  65  157   32    7  13  94.0  24.0  0.0  67  65.0   
          1916 NL      53   4    6    1    0   1   8.0   0.0  0.0   4   6.0   
zychto01  2015 AL       0   0    0    0    0   0   0.0   0.0  0.0   0   0.0   
          2016 AL       0   0    0    0    0   0   0.0   0.0  0.0   0   0.0   
          2017 AL       0   0    0    0    0   0   0.0   0.0  0.0   0   0.0   

                      ibb  hbp    sh   sf  gidp  
player_id yr   lg_id                             
aardsda01 2004 NL     0.0  0.0   0.0  0.0   0.0  
          2006 NL     0.0  0.0   1.0  0.0   0.0  
          2007 AL     0.0  0.0   0.0  0.0   0.0  
          2008 AL     0.0  0.0   0.0  0.0   0.0  
          2009 AL     0.0  0.0   0.0  0.0   0.0  
...                   ...  ...   ...  ...   ...  
zwilldu01 1915 FL     0.0  2.0  18.0  0.0   0.0  
          1916 NL     0.0  0.0   2.0  0.0   0.0  
zychto01  2015 AL     0.0  0.0   0.0  0.0   0.0  
          2016 AL     0.0  0.0   0.0  0.0   0.0  
          2017 AL     0.0  0.0   0.0  0.0   0.0  

[102806 rows x 16 columns]

In [3]:
ppl = bbl.load_people()[['player_id', 'bats', 'throws']]
ppl

,player_id,bats,throws
0,aardsda01,R,R
1,aaronha01,R,R
2,aaronto01,R,R
3,aasedo01,R,R
4,abadan01,L,L
...,...,...,...
20088,zupofr01,L,R
20089,zuvelpa01,R,R
20090,zuverge01,R,R
20091,zwilldu01,L,L


In [4]:
hr_bat = bsu.add_batting_rate_stats(bat)[['pa', 'hr']] # add_batting_rate_stats will compute pa
bat_seasons = pd.merge(left=hr_bat.reset_index(), right=ppl, on='player_id')
bat_seasons

,player_id,yr,lg_id,pa,hr,bats,throws
0,aardsda01,2004,NL,0.0,0,R,R
1,aardsda01,2006,NL,2.0,0,R,R
2,aardsda01,2007,AL,0.0,0,R,R
3,aardsda01,2008,AL,1.0,0,R,R
4,aardsda01,2009,AL,0.0,0,R,R
...,...,...,...,...,...,...,...
102801,zwilldu01,1915,FL,617.0,13,L,L
102802,zwilldu01,1916,NL,57.0,1,L,L
102803,zychto01,2015,AL,0.0,0,R,R
102804,zychto01,2016,AL,0.0,0,R,R


In [5]:
bat_totals = bat_seasons.groupby(['yr', 'bats'])[['pa', 'hr']].sum()
bat_rolling_totals = bat_totals.unstack(level=1).rolling(5).sum().dropna()
bat_rolling_totals


pa                          hr                 
bats         B         L         R       B       L        R
yr                                                         
1880    2947.0   18987.0   69396.0     3.0    64.0    135.0
1881    4000.0   22685.0   70565.0     6.0    74.0    157.0
1882    5191.0   29941.0   85256.0    11.0   118.0    241.0
1883    7109.0   40651.0  113088.0    15.0   170.0    390.0
1884   11492.0   57677.0  162822.0    36.0   298.0    831.0
...        ...       ...       ...     ...     ...      ...
2016  119532.0  309459.0  485623.0  2615.0  8119.0  13566.0
2017  119198.0  305273.0  491793.0  2731.0  8419.0  14321.0
2018  119689.0  299101.0  498284.0  2908.0  8501.0  14986.0
2019  118145.0  295658.0  506388.0  3168.0  9292.0  16525.0
2020  103387.0  256566.0  444188.0  2924.0  8344.0  15112.0

[141 rows x 6 columns]

In [6]:
bat_rolling_rates = bat_rolling_totals['hr']/bat_rolling_totals['pa']
bat_rolling_rates

bats,B,L,R
yr,,,
1880,0.001018,0.003371,0.001945
1881,0.001500,0.003262,0.002225
1882,0.002119,0.003941,0.002827
1883,0.002110,0.004182,0.003449
1884,0.003133,0.005167,0.005104
...,...,...,...
2016,0.021877,0.026236,0.027935
2017,0.022911,0.027579,0.029120
2018,0.024296,0.028422,0.030075


In [7]:
fig = px.line(bat_rolling_rates[['L', 'R', 'B']].stack().rename('rate').reset_index(), x='yr', y='rate', color='bats')
fig.update_layout(width=800)
fig.write_image('output/hr_rates_by_handedness.png')
fig.show()

In [8]:
bat_rolling_ratios = (bat_rolling_rates['L']/bat_rolling_rates['R']).rename('HR Rate LHB:RHB')
bat_rolling_ratios

yr
1880    1.732704
1881    1.466164
1882    1.394195
1883    1.212634
1884    1.012338
          ...   
2016    0.939176
2017    0.947068
2018    0.945025
2019    0.963078
2020    0.955917
Name: HR Rate LHB:RHB, Length: 141, dtype: float64

In [9]:
fig = px.line(bat_rolling_ratios)
fig.update_layout(width=800)
fig.write_image('output/hr_rates_by_handedness_ratio.png')
fig.show()

In [10]:
# Out of curiosity, how much did the mix of playing time by handedness change
pa_totals = bat_totals['pa'].unstack(level=1)
pa_totals_rolling = pa_totals.rolling(5).sum().dropna()
pa_sum_rolling = pa_totals_rolling.sum(axis=1)
pa_fractions = pa_totals_rolling.divide(pa_sum_rolling, axis=0)

fig = px.line(pa_fractions[['L', 'R', 'B']])
fig.update_layout(width=800)
fig.write_image('output/pa_by_handedness.png')
fig.show()


In [11]:
# Is there any NL/AL effect?  Look at LHB/RHB ratios for each league
alnl = bat_seasons[(bat_seasons['lg_id'].isin(['AL', 'NL']))&(bat_seasons['yr']>1900)]
bat_totals = alnl.groupby(['yr', 'bats', 'lg_id'])[['pa', 'hr']].sum()
bat_rolling_totals = bat_totals.unstack().unstack().rolling(5).sum().dropna()
bat_rolling_rates = bat_rolling_totals['hr']/bat_rolling_totals['pa']
bat_rolling_rates

lg_id        AL                            NL                    
bats          B         L         R         B         L         R
yr                                                               
1905   0.003524  0.004482  0.005025  0.003330  0.004722  0.003478
1906   0.002859  0.004100  0.004482  0.002370  0.003959  0.003211
1907   0.002105  0.003395  0.003611  0.002256  0.004064  0.003489
1908   0.002045  0.003143  0.003125  0.002379  0.003879  0.003484
1909   0.001640  0.002993  0.002885  0.002409  0.003752  0.003335
...         ...       ...       ...       ...       ...       ...
2016   0.023083  0.027624  0.030111  0.020085  0.024904  0.025983
2017   0.024244  0.028516  0.031142  0.020892  0.026699  0.027196
2018   0.025495  0.028921  0.032073  0.022632  0.027971  0.028103
2019   0.027564  0.032090  0.034722  0.025818  0.030841  0.030533
2020   0.028721  0.033107  0.035696  0.027723  0.032014  0.032301

[116 rows x 6 columns]

In [12]:
# Visualize each league's trend (I tried putting it all on one chart, but it was too busy)
for lg in ('AL', 'NL'):
    fig = px.line(bat_rolling_rates[lg][['L', 'R']], labels={'value': 'HR rate (5-year rolling)'})
    fig.update_layout(width=800, title=f"HR Rate by handedness ({lg})")
    fig.write_image(f'output/hr_rates_by_handedness_{lg}.png')
    fig.show()

# Looks like the LH/RH effect from 1910-1950 was strong in both leagues

In [13]:
# OK, let's look at the distribution of HR hitters.  Look by cohort, etc

# This will filter the sample by handedness and then split into deciles
# Used by compare_deciles()
def get_deciles(exploded, bats):
    df = exploded.query('bats==@bats')
    bins = pd.qcut(df['hr_rate'],10)
    return(df.groupby(bins)['hr_rate'].mean())

# This will generate the decile summary for a given set of years
def compare_deciles(years):
    # Get the raw stats, then add PA, handedness and HR rate
    raw = bbl.load_batting(years, coalesce_type=bbl.CoalesceMode.PLAYER_CAREER)
    hr_bat = bsu.add_batting_rate_stats(raw)[['pa', 'hr']] # add_batting_rate_stats will compute pa
    bat_careers = pd.merge(left=hr_bat.reset_index(), right=ppl, on='player_id')
    bat_careers['hr_rate'] = bat_careers['hr']/bat_careers['pa']
    
    # To weight by PA, repeat each line once per PA
    exploded = pd.DataFrame(bat_careers.values.repeat(bat_careers['pa'], axis=0), columns=bat_careers.columns)[['player_id', 'bats', 'hr_rate']]
    exploded['hr_rate'] = exploded['hr_rate'].apply(float)

    # Now compute the summary means
    by_handed = pd.concat([get_deciles(exploded, bats).rename(bats).reset_index(drop=True) for bats in ('L', 'R')], axis=1)
    return by_handed

deciles = compare_deciles(range(1910, 1951))
deciles


,L,R
0,0.000791,0.000061
1,0.003121,0.001428
2,0.004814,0.002791
3,0.007079,0.004115
4,0.009219,0.005769
5,0.011081,0.007572
6,0.013414,0.010087
7,0.017638,0.014117
8,0.025321,0.019820
9,0.041561,0.035557


In [14]:

for yrs in (range(1910, 1947), range(1947, 1970), range(1970, 2021)):
    comparison = compare_deciles(yrs)
    fig = px.line(comparison, labels={'value': f'Cohort Career HR Rate', 'index': 'decile'})
    fig.update_layout(width=800, title=f"HR Rate by handedness and decile ({yrs.start}-{yrs.stop-1})")
    fig.write_image(f'output/hr_rates_by_cohort_{yrs.start}-{yrs.stop-1}.png')
    fig.show()


In [15]:
deciles['L']/deciles['R']

0    12.979503
1     2.185879
2     1.724809
3     1.720154
4     1.597909
5     1.463341
6     1.329888
7     1.249365
8     1.277554
9     1.168840
dtype: float64